In [58]:
# load the packages 
import pickle
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import LabelEncoder

# Load required pkl and csv

In [107]:
# group1 = pd.read_csv("ColumnSubsets\FAERS_columnSubset_Group_1.csv")
# group2 = pd.read_csv("ColumnSubsets\FAERS_columnSubset_Group_2.csv")
# group3 = pd.read_csv("ColumnSubsets\FAERS_columnSubset_Group_3.csv")
# group4 = pd.read_csv("ColumnSubsets\FAERS_columnSubset_Group_4.csv")
# group5 = pd.read_csv("ColumnSubsets\FAERS_columnSubset_Group_5.csv")
# group6 = pd.read_csv("ColumnSubsets\FAERS_columnSubset_Group_6.csv")
# group7 = pd.read_csv("ColumnSubsets\FAERS_columnSubset_Group_7.csv")
# group8 = pd.read_csv("ColumnSubsets\FAERS_columnSubset_Group_8.csv")
group9 = pd.read_csv("ColumnSubsets\FAERS_columnSubset_Group_9.csv")
#group10 = pd.read_csv("ColumnSubsets\FAERS_columnSubset_Group_10.csv")

In [15]:
# load the unique_dict_categorical file for unique values for the feature matrix 
file_name = open("unique_categorical_dict.pkl", "r")
unique_categorical_dict = pickle.load(file_name)
file_name.close()

In [16]:
# load the  file for unique values for the reaction
unique_react = pd.read_csv("distinct_reactions.csv")
unique_react_list = unique_react['class'].unique()

# OneHotEncoding and split the feature matrix(X) and reactions(Y)

In [106]:
#Create function to convert to dummy variables
def convert_to_dummies(data, categorical_columns, unique_dictionary):
    #Assign the categories for each column
    for column in categorical_columns:
        data[column] = data[column].astype('category', categories=unique_dictionary[column])
    #Convert to categorical
    dummy = pd.get_dummies(data, columns=categorical_columns)
    return dummy

In [108]:
# Code to convert to dummy a specific group, only the feature matrix, the reaction will keep the same 
categorical_columns = ['RxCUI', 'indications', 'role_cod', 'sex', 'age_grp', 
                       'dechal', 'rechal']
# group1_dummy = convert_to_dummies(group1, categorical_columns, unique_categorical_dict)
# group2_dummy = convert_to_dummies(group2, categorical_columns, unique_categorical_dict)
# group3_dummy = convert_to_dummies(group3, categorical_columns, unique_categorical_dict)
#group4_dummy = convert_to_dummies(group4, categorical_columns, unique_categorical_dict)
# group5_dummy = convert_to_dummies(group5, categorical_columns, unique_categorical_dict)
# group6_dummy = convert_to_dummies(group6, categorical_columns, unique_categorical_dict)
# group7_dummy = convert_to_dummies(group7, categorical_columns, unique_categorical_dict)
# group8_dummy = convert_to_dummies(group8, categorical_columns, unique_categorical_dict)
group9_dummy = convert_to_dummies(group9, categorical_columns, unique_categorical_dict)
#group10_dummy = convert_to_dummies(group10, categorical_columns, unique_categorical_dict)

In [19]:
# Separate select the reactions columns to create Y_value
def separate_x_y(data):
    data_x = data[data.columns.difference(['Unnamed: 0','Unnamed: 0.1','caseid','drug_seq','reactions'])]
    data_y =data.filter(regex='reactions')
    return data_x,data_y

In [109]:
# create the oneHotEncoding data
# group1_x,group1_y = separate_x_y(group1_dummy)
# group2_x,group2_y = separate_x_y(group2_dummy)
# group3_x,group3_y = separate_x_y(group3_dummy)
# group4_x,group4_y = separate_x_y(group4_dummy)
# group5_x,group5_y = separate_x_y(group5_dummy)
# group6_x,group6_y = separate_x_y(group6_dummy)
# group7_x,group7_y = separate_x_y(group7_dummy)
# group8_x,group8_y = separate_x_y(group8_dummy)
group9_x,group9_y = separate_x_y(group9_dummy)
#group10_x,group10_y = separate_x_y(group10_dummy)

# Reshape both the X and Y, Save all as Pickled Files

In [50]:
# Reshape the Feature Matrix 
# the funcation to reshape the data format, size is the number of the group, group is the data
def splitX(size, group):
    caseid_data = []
    group_size = len(group)/size
    for case_order in range(group_size):
        row = group.loc[(case_order*size):(case_order*size+(size-1))].values
        caseid_data.append(row)
    return caseid_data

# Reshape the Y table
def split_coma(s):
    return s.split('\\,')

def splitY(size,group):
    reactions_splited = map(split_coma, group['reactions'])
    mlb = MultiLabelBinarizer(classes=unique_react_list)  # oneHotEncoding function
    #Transform on each group
    group_formated = mlb.fit_transform(reactions_splited)
    print (mlb.inverse_transform(group_formated))
    caseid_data = []
    group_size = len(group_formated)/size
    for case_order in range(group_size):
        row = group_formated[(case_order*size):(case_order*size+size)]
        caseid_data.append(row)
    return caseid_data

def savePickledFile(name,data):
    with open (name,"wb") as fp:
        pickle.dump(data,fp)

In [121]:
# Save all the feature matrix for different groups into pickled File 
# group1_x_formated = splitX(1,group1_x)
# savePickledFile("group1_x_formated.pkl",group1_x_formated)
# group1_y_formated = splitY(1,group1_y)
# savePickledFile("group1_y_formated.pkl",group1_y_formated)    

# group2_x_formated = splitX(2,group2_x)
# savePickledFile("group2_x_formated.pkl",group2_x_formated)
# group2_y_formated = splitY(2,group2_y)
# savePickledFile("group2_y_formated.pkl",group2_y_formated)  

# group3_x_formated = splitX(3,group3_x)
# savePickledFile("group3_x_formated.pkl",group3_x_formated)
# group3_y_formated = splitY(3,group3_y)
# savePickledFile("group3_x_formated.pkl",group3_x_formated)  

# group4_x_formated = splitX(4,group4_x)
# savePickledFile("group4_x_formated.pkl",group4_x_formated)
# group4_y_formated = splitY(4,group4_y)
# savePickledFile("group4_y_formated.pkl",group4_y_formated)  

# group5_x_formated = splitX(5,group5_x)
# savePickledFile("group5_x_formated.pkl",group5_x_formated)
# group5_y_formated = splitY(5,group5_y)
# savePickledFile("group5_y_formated.pkl",group5_y_formated)  

# group6_x_formated = splitX(6,group6_x)
# savePickledFile("group6_x_formated.pkl",group6_x_formated)
# group6_y_formated = splitY(6,group6_y)
# savePickledFile("group6_y_formated.pkl",group6_y_formated) 

# group7_x_formated = splitX(7,group7_x)
# savePickledFile("group7_x_formated.pkl",group7_x_formated)
# group7_y_formated = splitY(7,group7_y)
# savePickledFile("group7_y_formated.pkl",group7_y_formated) 

# group8_x_formated = splitX(8,group8_x)
# savePickledFile("group8_x_formated.pkl",group8_x_formated)
# group8_y_formated = splitY(8,group8_y)
# savePickledFile("group8_y_formated.pkl",group8_y_formated) 

# group9_x_formated = splitX(9,group9_x)
# savePickledFile("group9_x_formated.pkl",group9_x_formated)
# group9_y_formated = splitY(9,group9_y)
# savePickledFile("group9_y__formated.pkl",group9_y_formated)

group10_x_formated = splitX(10,group10_x)
savePickledFile("group10_x_formated.pkl",group10_x_formated)
group10_y_formated = splitY(10,group10_y)
savePickledFile("group10_y_formated.pkl",group10_y_formated)

In [110]:
reactions_splited = map(split_coma, group9_y['reactions'])
mlb = MultiLabelBinarizer(classes=unique_react_list)  # oneHotEncoding function
    #Transform on each group
group_formated = mlb.fit_transform(reactions_splited)

In [94]:
#print(unique_react_list.dtype.names)
print (mlb.inverse_transform(group_formated)[1])


('PAPILLARY THYROID CANCER', 'GOITRE')


In [101]:
np.where(group_formated[1]==1)

(array([4069, 7273], dtype=int64),)

In [103]:
print (list(mlb.classes_)[7273])

GOITRE
